In [9]:
import bson
import brtdevkit
from brtdevkit.core.db import DBConnector
from brtdevkit.core.db import ObjectIdFilter
from brtdevkit.core.db.db_filters import *
import brtdevkit.util.s3 as brt_s3

import folium
import pygeohash as gh
import re

import pathlib
import json

import time
from datetime import date, datetime

import numpy as np
import pandas as pd

import webbrowser
from random import shuffle

In [2]:
def get_ultra_sparse(start_time, end_time, robot):
    
    robot_names = ['db1', 'db2', 'db3', 'db4', 'db5', 'db6', 'db7', 'db8', 'db9', 'db10', 'DB1', 'DB2', 'DB3', 'DB4', 'DB5', 'DB6', 'DB7', 'DB8', 'DB9']
    img_filters = [
        {# {"$in": robot_names},
            'project_name': 'shasta',
            'robot_name': robot,
            'active_model': {'$exists': True},
            'artifacts': {"$elemMatch": {'s3_bucket': {"$exists": True}, 's3_key': {"$exists": True}}}
        }, 
        DatetimeFilter(key='collected_on', start=start_time, end=end_time),
    ]
    connector = DBConnector()
    df = connector.get_documents_df('image', img_filters, limit=None)
    print("I have a dataframe of length: ", len(df))
    return df
    
def add_geohash(df, precision):
    
    geohash_cp = df.copy()
    geohash_cp['geohash'] = df.apply(
        lambda x: gh.encode(x.latitude, x.longitude, precision=precision), axis=1)
    
    return geohash_cp

def get_crop(df_row):
    with open(f"{df_row.active_model}_metadata.json") as f:
          metadata = json.load(f)
    crop = metadata['crop_type'][0].upper()
    return crop

def get_prod_model_local(df_row):
    crop = df_row['crop_name']
    local_path = f"{crop}_model.jit"
    return local_path

def get_visual_df(image_initial):
    visual_df = image_initial[['_id','latitude', 'longitude', 'collected_on', 'robot_name']]
    
    # make a column for image id, with the type string
    # currently it is of the form objectID, which will cause issues later
    visual_df['img_id'] = visual_df['_id'].astype('str')
    
    # Create a link using the image id
    # dp_id should be changed to _id if working with the image dataframe
    visual_df['link'] = "https://aletheia.brtws.com/images/"+visual_df["img_id"]
    
    # Make the link clickable
    link_text = str(visual_df['link'])
    visual_df['test_link'] = folium.Html('<a href="' + visual_df['link'] +'"target="_blank">' + re.sub(r"[']+", "\\\\'", link_text[:45]) + '</a>', script=True)
    
    # Add it as a pop-up
    visual_df['popup'] = folium.Popup(visual_df['test_link']) 
    return visual_df

def get_map(visual_df, zoom_start=17, show_all=False):
    # Create a list of locations for plotting using the gps coordinates
    locations = visual_df[['latitude', 'longitude']]
    
    locationlist = locations.values.tolist()
    # create a base map
    # leave the zoom number low, e.g. 5 if you want a view of half the county
    # if you just want an area to cover a part of the state 10 is probably a good number
    # 13 is a good number if you just want to look at a field 
    # control scale adds a scale bar 
    map = folium.Map(location=locationlist[0], zoom_start=zoom_start, control_scale=True, max_zoom=26);

    # Make the link clickable
    link_text = str(visual_df['link'])
    if show_all == True:
        for i in visual_df.index:
            webbrowser.open_new(visual_df['link'][i]) 
    # add data points to map
    # we are adding clickable links to the points 
    # these links will direct you back to the original image in aletheia 
    for point in range(0, len(locationlist)):
        #print(locationlist[point])
        #print(visual_df['link'][point])
        
        # changed to location_list
        folium.Circle(locationlist[point], 
                      popup=folium.Popup(folium.Html('<a href="' + visual_df['link'].iloc[point] + '"target="_blank">' + re.sub(r"[']+", "\\\\'", link_text[:45]) +'</a>', script = True)), 
                      color = 'coral', radius = 0.3).add_to(map);
        
    return map

def add_satellite_overlay(map):
    # Create an overlay on satellite map
    tile = folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = False,
            control = True
           ).add_to(map)

def add_text(img, text):
    draw = ImageDraw.Draw(img)
    # draw.text((x, y),"Sample Text",(r,g,b))
    draw.text((50, 0), text, (255,255,255))
    
def sample_n(df, n=25):
    df= df[(df['angle_to_row']=="PARALLEL")]
    shuffled_index = random.sample(list(df._id), n)
    sampled_df = df[df._id.isin(shuffled_index)]
    return sampled_df

def generate_overlay_image(df, zoom=16, show_all=False, sample=False):
    """
    Enter the dataframe name and the robot_name to subset. 
    """
    image_initial = df
    print(len(image_initial))
    if sample == True:
        visual_df = get_visual_df(sample_n(image_initial))
    else:
        visual_df = get_visual_df(image_initial)
    map = get_map(visual_df, zoom_start=zoom, show_all=show_all)
    add_satellite_overlay(map)
    folium.features.ClickForMarker(popup=None).add_to(map)
    #img = grab_img(map)
    print('\n')
    return map  


### Get USL data 

In [3]:
# Start and end dates of the data we care about
start_date =datetime(2021,5, 1, 1, 0)
end_date =datetime(2021, 7, 30, 1, 1)
robot_name = 'db3'

usl_df = get_ultra_sparse(start_date, end_date, robot=robot_name)
#usl_df = add_geohash(usl_df, 6)

# Add a date to the DataFrame
usl_df['date_collected'] = pd.to_datetime(usl_df['collected_on']).dt.date.astype(str)
usl_df.robot_name.value_counts()

I have a dataframe of length:  16562


db3    16562
Name: robot_name, dtype: int64

### Group by Robt Name, Date and Crop (Active Model)

In [5]:
# If the machine is running one of these models then we're probably not interested in it's data
exclude_models = ['20201202_3_tritonstgold_5', '20210415_1_flag_4', '20210511_1_flag_4', '20210112_1_qrcode_4']

usl_events = usl_df[~usl_df['active_model'].isin(exclude_models)].groupby(['robot_name', 'date_collected', 'active_model']).size().reset_index()
usl_events = usl_events.sort_values(by='date_collected').reset_index()
usl_locations  = usl_df[~usl_df['active_model'].isin(exclude_models)].groupby(['robot_name', 'date_collected','latitude', 'active_model','longitude']).mean().reset_index()
usl_locations.head()

,robot_name,date_collected,latitude,active_model,longitude,is_deleted,has_human_annotation,camera_exposure_time,camera_focal_length,camera_fstop,camera_gain,camera_height,camera_index,camera_ppi,gps_course,gps_speed,raw_bit_depth,slot_id
0,db3,2021-06-05,35.769726,20210330_1_soybeans_4,-89.071190,False,False,1500.0,3.6,1.8,1.0,1.569024,1.5,18.342343,4.783431,2.649742,20.0,1.0
1,db3,2021-06-05,35.769726,20210330_1_soybeans_4,-89.071182,False,False,1500.0,3.6,1.8,1.0,1.574655,1.5,18.230682,4.731965,2.631295,20.0,3.0
2,db3,2021-06-05,35.769726,20210330_1_soybeans_4,-89.071175,False,False,1500.0,3.6,1.8,1.0,1.684659,1.5,17.045369,4.672480,2.635636,20.0,12.0
3,db3,2021-06-05,35.769993,20210330_1_soybeans_4,-89.071075,False,False,1500.0,3.6,1.8,1.0,1.489685,1.5,19.294832,6.260235,0.001085,20.0,2.0
4,db3,2021-06-05,35.770103,20210330_1_soybeans_4,-89.071487,False,False,1500.0,3.6,1.8,1.0,1.600237,1.5,17.938839,6.098436,3.254126,20.0,13.0


In [7]:
robot_loc = usl_locations[['robot_name','active_model' , 'date_collected','latitude', 'longitude']]
robot_locs = robot_loc.groupby(['robot_name', 'active_model','date_collected']).mean().reset_index()
robot_locs = robot_locs.sort_values('date_collected').reset_index()
robot_locs

,index,robot_name,active_model,date_collected,latitude,longitude
0,0,db3,20210330_1_soybeans_4,2021-06-05,35.772701,-89.072252
1,4,db3,20210524_1_cotton_4,2021-06-05,35.834019,-89.326877
2,1,db3,20210330_1_soybeans_4,2021-06-07,35.831303,-89.329262
3,5,db3,20210524_1_cotton_4,2021-06-07,35.832556,-89.328447
4,2,db3,20210330_1_soybeans_4,2021-06-15,35.752811,-89.788200
5,6,db3,20210524_1_cotton_4,2021-06-15,35.754814,-89.787416
6,7,db3,20210524_1_cotton_4,2021-06-16,35.717711,-89.894979
7,8,db3,20210524_1_cotton_4,2021-06-17,35.716704,-89.901159
8,9,db3,20210524_1_cotton_4,2021-06-19,35.644313,-89.782014
9,3,db3,20210330_1_soybeans_4,2021-06-22,35.749065,-89.800100


### Look At Spatial distribution in Sattelite Overlay

In [8]:
# Choose an index from robot_locs to examine
n = 12

image_df = usl_df[(usl_df['robot_name'] == robot_locs['robot_name'][n]) & 
                  (usl_df['date_collected'] == robot_locs['date_collected'][n]) & 
                  (usl_df['active_model'] == robot_locs['active_model'][n])]
                  
images = list(image_df['_id'])
shuffle(images)

# Opens randomly chosen images in new tabs
for i in range(10):
    webbrowser.open_new_tab('https://aletheia.brtws.com/images/' + images[i])
    
print(image_df.active_model.unique())
print(image_df.date_collected.unique())
print(image_df.node.value_counts())
generate_overlay_image(df = image_df)

['20210524_1_cotton_4']
['2021-06-24']
vpu0-3a     724
vpu0-1a     616
vpu0-10a    532
vpu0-2a     396
vpu0-13a    252
vpu0-12a    144
vpu0-11a     48
Name: node, dtype: int64
2712


/home/williamroberts/code/brtdevkit/venv/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/williamroberts/code/brtdevkit/venv/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/williamroberts/code/brtdevkit/venv/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value